In [ ]:
import sys
import os
_path = os.path.abspath('../scripts/')
if _path not in sys.path:
    sys.path.append(_path)

import astropy.coordinates as coord
coord.galactocentric_frame_defaults.set('v4.0')
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from scipy.special import logsumexp

from pyia import GaiaData
import theano
# theano.config.gcc.cxxflags = "-fbracket-depth=16000" # default is 256 - only need this on Mac??
import pymc3 as pm
import theano.tensor as tt
import exoplanet as xo

from model import GroupHepler

In [ ]:
run_name = '400pc-cube-result'
t = at.Table.read(f'../cache/probs-{run_name}.fits')
d = at.Table.read(f'../data/{run_name}.fits.gz')
joined = at.join(d, t, keys='source_id', join_type='left')

allg = GaiaData(joined)
g = allg[(allg.prob > 0.5) & (allg.get_ruwe() < 1.4)]

In [ ]:
clean_gal = g.get_skycoord().galactic
clustered = ((clean_gal.cartesian.x < 50*u.pc) &
             (clean_gal.cartesian.x > -5*u.pc) &
             (clean_gal.cartesian.y > 0*u.pc) &
             (clean_gal.cartesian.y < 50*u.pc) &
             (clean_gal.cartesian.z > 30*u.pc) &
             (clean_gal.cartesian.z < 50*u.pc))
clean_mg = (g.phot_g_mean_mag - g.distmod)
da_best = g[clustered & (clean_mg < 8*u.mag)]

len(da_best)

In [ ]:
da_best.data.write('../data/the-best-16.csv', overwrite=True)

### Take a 200 pc cube centered on this clump:

In [ ]:
best_xyz = da_best.get_skycoord().galactic.cartesian.without_differentials()
best_xyz = best_xyz.mean()

In [ ]:
all_xyz = allg.get_skycoord().galactic.cartesian.without_differentials()
og_200pc_cube_mask = np.all(np.abs((all_xyz - best_xyz).xyz) < 100*u.pc, 
                            axis=0)
og_200pc_cube_mask.sum()

In [ ]:
allg.data[og_200pc_cube_mask].write('../data/clump-200pc-cube.fits', overwrite=True)

### Run the Group model on the best members:

In [ ]:
helper = GroupHelper(da_best)

In [ ]:
model = helper.get_model()

In [ ]:
with model:
    res = xo.optimize(start={'vxyz': helper.test_vxyz,
                             'r': helper.test_r})

In [ ]:
res['sigv']

In [ ]:
res['meanv']

In [ ]:
with model:
    trace = pm.sample(start=res,
                      tune=2000,
                      draws=1000,
                      cores=1, chains=2)

In [ ]:
names = ['x', 'y', 'z']

In [ ]:
v0 = [-6.932, 24.301, -9.509]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, ax in enumerate(axes):
    ax.hist(trace['meanv'][:, i])
    ax.set_xlabel(f'$v_{names[i]}$ [km/s]')
    ax.axvline(v0[i], color='tab:red')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, ax in enumerate(axes):
    ax.hist(trace['sigv'][:, i])
    ax.set_xlabel(f'$\sigma_{{v_{names[i]}}}$ [km/s]')
    ax.axvline(0.6, color='tab:red')

In [ ]:
np.median(trace['meanv'], axis=0)

In [ ]:
np.median(trace['sigv'], axis=0)